In [15]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import metrics
import dill as pickle

In [16]:
df_products= pd.read_csv("new.csv")

In [17]:
df_products.head()

,name,description
0,Airpods Wireless Bluetooth Headphones,Bluetooth technology lets you connect it with ...
1,iPhone 11 Pro 256GB Memory,Introducing the iPhone 11 Pro. A transformativ...
2,Cannon EOS 80D DSLR Camera,"Characterized by versatile imaging specs, the ..."
3,Sony Playstation 4 Pro White Version,The ultimate home entertainment center starts ...
4,Logitech G-Series Gaming Mouse,Get a better handle on your games with this Lo...


In [18]:
df_products['description']

0    Bluetooth technology lets you connect it with ...
1    Introducing the iPhone 11 Pro. A transformativ...
2    Characterized by versatile imaging specs, the ...
3    The ultimate home entertainment center starts ...
4    Get a better handle on your games with this Lo...
5    Meet Echo Dot - Our most popular smart speaker...
Name: description, dtype: object

In [19]:
tfidf = TfidfVectorizer(stop_words="english")
df_products['description'] = df_products['description'].fillna("")
tfidf_matrix = tfidf.fit_transform(df_products['description'])

In [20]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [21]:
indices = pd.Series(df_products.index, index=df_products['name']).drop_duplicates()

In [22]:
indices['Sony Playstation 4 Pro White Version']

3

In [27]:
  def get_recommendations(name, cosine_sim = cosine_sim):
      idx = indices[name]
      sim_scores = enumerate(cosine_sim[idx])
      sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
      sim_scores = sim_scores[1:11]
    
      sim_index = [i[0] for i in sim_scores]
  
      y = df_products['name'].iloc[sim_index]
      return y

      
      

  get_recommendations('Sony Playstation 4 Pro White Version')  

4           Logitech G-Series Gaming Mouse
0    Airpods Wireless Bluetooth Headphones
1               iPhone 11 Pro 256GB Memory
2               Cannon EOS 80D DSLR Camera
5           Amazon Echo Dot 3rd Generation
Name: name, dtype: object

In [26]:
model = {
    'tfidf': tfidf,
    'cosine_sim': cosine_sim,
    'indices': indices,
    'get_recommendations': get_recommendations,
    'df_products': df_products
}
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)